# <font color='056938'> **Load modules** </font>

We will load into this notebook some modules with functions that will help you in the development of heuristics and metaheuristics for the solution of travelling salesman problemas

The module `random_generators.py` include the functions to deal with generating random numbers. The ultimate function of this module is to generate a random permutation (solution) of the tsp problem
* Random tour generation: `rand_permutation`

The module `tsp_tools` provides different functions to help from the data reading to the solution ploting
* Data reading: `read_data(filepath)`
> `filepath`: the route to the file with the instance's data
* Distance calculation: `calculate_distances`
* Data display: `print_TSPdata()
* Solution evaluation: `tsp_distance`
* Tour display: `plot_tsp_route()`
* Plot the search trayectory `plot_trace(trace)`

In [ ]:
!gdown 1Z81jdqyAq3kX9xbZWGgOwV2UHpGDbUbq
!gdown 1xnT-vDHdSbXVAp-exyAFR8rxVSD_moIa

import random_generators as ran_gen
import tsp_tools as tools

# <font color='056938'> **Instance generation** </font>

## <font color='8EC044'> **Data reading** </font>

We will use the clasical test instances stored in  [TSPlib](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/), the format of these instances include a heading with the following information:

>NAME: berlin52

>TYPE: TSP

>COMMENT: 52 locations in Berlin (Groetschel)

>DIMENSION: 52

>EDGE_WEIGHT_TYPE: EUC_2D

>NODE_COORD_SECTION

Then, n lines with the coordinates of the cities in the following format (cityID, Coordinate X, Coordinate Y):

**id cX cY**

Knowing this structure we are going to read two files

* `berlin52.tsp`
* `prueba10.tsp`



First we load the datafile to the colab environment

In [ ]:
!gdown 1ZvapfCjHHfEpD-vbOgD92obxi36bnS91
!gdown 16AEMPDJQQDnyJAGsWaPnNjXUz6u5EbVa

Then we read the data file

In [ ]:
data = tools.read_data("prueba10.tsp")             #Read the data
data

We get the size of the instance. That is, the number of nodes `n`

In [ ]:
n = len(data)
n

## <font color='8EC044'> **Distance calculation** </font>

We are going to use the $\tt{scipy.spatial}$ package to calculate the distances trhough the function `calculate_distances()`



In [ ]:
distances=tools.calculate_distances(data)
distances

# <font color='056938'> **1. Swarm Intelligence** </font>

In the field of metaheuristics, Swarm Intelligence refers to the development and investigation of effective computational techniques for problem-solving, drawing inspiration from the collective behavior of real swarms and insect colonies [Merkle & Middendorf, 2001](https://link.springer.com/chapter/10.1007/0-387-28356-0_14). Several methods based on swarm intellince have been sucessfully applyied for solving optimization problems, among which the ant colony optimization (ACO) and particle swarm optimization (PSO) stand out.

In this notebook, we will delve into the study of the ACO method gaining an understanding of the key components required for their implementation.

## <font color='056938'> **1.1 Initial solution generation** </font>

As usual, we will use the Traveling Salesman Problem (TSP) as a toy problem for illustrate the functionality of these methods. The code in the modules we load makes use of the code from [Taillard](http://mistic.heig-vd.ch/taillard/articles.dir/taillard.html) (2023), which is, in turn, built upon [L'Ecuyer's](https://www.iro.umontreal.ca/~lecuyer/) random number generator for randomness.

It is used to generate a random solution. That is an inital random permutation:

In [ ]:
sol=ran_gen.rand_permutation(n)
sol

##<font color='056938'> **1.1.1. Solution evaluation** </font>

### <font color='8EC044'> **Objective function calculation** </font>

Now we define use function `tsp_distance()` to evaluate the objective function of a given permutation

For the case of the solution we previuosly generated it will be:

In [ ]:
obj_fn = tools.tsp_distance(distances,sol)
print("Distance of the permutation --> " + str(obj_fn))

## <font color='056938'> **1.2 Ant Colony Optimization** </font>

Ant Colony Optimization (ACO) is a nature-inspired optimization algorithm that is based on the foraging behavior of ants. It is commonly used to find solutions to complex combinatorial optimization problems. In ACO, a population of artificial ants iteratively builds solutions to a problem by depositing and following pheromone trails. Ants communicate through these pheromone trails, reinforcing paths that lead to good solutions and fading away those that lead to suboptimal ones. Over time, the algorithm converges towards an optimal or near-optimal solution.

![](https://drive.google.com/uc?id=1u29ho_diexfgCToEJx2J61lDEJft-2Pm)

This is the basic algorithm to implement an ACO method for solving TSP:

![](https://drive.google.com/uc?id=16RP341x9rLbhR95j7I1V6kibJGYRmYtL)

Let's take a look to the main components needed to implement this algorithm.

### <font color='8EC044'> **1.2.1 How to deﬁne the trails of an artiﬁcial ant colony for TSP?** </font>



Modeling pheromone trails is a crucial aspect of the algorithm, as it guides the artificial ants in their search for solutions. For TSP, the pheromone information for permutation problems can usually be encoded in
an $n × n$ pheromone matrix $[\tau_{i,j}], i,j \in [1:n]$. The pheromone value $\tau_{i,j}$ expresses the desirability to assign city $j$ after city $i$ in the permutation. The pheromone matrix for the TSP problem is initialized so that all values $\tau_{i,j}$ with $i \neq j$ are the same.

Using this model, the ACO algorithm for TSP will be the following:

![](https://drive.google.com/uc?id=1V-vcS6woNf6HSBUHhN_cRLpANpYO8LWu)

The first part of the algorithm is a constructive heuristic (red box), which selects the next city from a probability $p_{ij}$. The second part (blue box) is the updating process for the pheromone trials (evaporation and intensification).

The following function will help us to initialize a pheromone trails matris:

In [ ]:
def init_trail(initial_value, trail):
  n = len(trail[0])
  for i in range(n):
    for j in range(n):
      trail[i][j] = initial_value

  for i in range(n):
    trail[i][i] = 0

  return trail

Let's define a pheromone matrix for the instance *prueba10.tsp*. It will be a matrix $10 \times 10$:

In [ ]:
n = 10
trail = [[-1] * n for _ in range(n)]
trail = init_trail(0.5, trail)
trail

At the begining all trails gets the same value, because we do not have posterior information about the problem.

The pheromone trails must updated after each iteration. This process is developed in two steps:

1. Evaporation: All pheromones trails are reduced by a fixed portion $ρ \in (0, 1)$:
$$\tau_{ij} := (1 - ρ) \cdot τ_{ij} ∀ i,j \in [1: n] $$
2. Intensification: All pheromone trails corresponding to the best solution $π^*$ are increased by an absolute amount $Δ > 0$:
$$\tau_{iπ^*} := τ_{i\pi^*} + \Delta  ∀ i \in [1: n] $$


Now, it is the time for you to program the `update_trails` function. This function receives the current pheromone trails matrix `trail`, the parameters `rho` and `delta`, and the permutation representing the best solution found in an iteration of the ACO algorithm.  

In [ ]:
def update_trails(trail, rho, delta, best_sol):
  n = len(trail)
  evaporation_factor = 1-rho
  #Evaporation

  #Intensification

  return trail

Let's test your implementation of the update process:

In [ ]:
n = 10
rho = 0.1
delta = 0.2
b_sol = [1,6,3,7,8,2,4,0,5,9]
t_test = [[-1] * n for _ in range(n)]
t_test = init_trail(0.5, t_test)
t_test = update_trails(t_test, rho, delta, b_sol)
t_test

This is the expected result:

```
[[0, 0.45, 0.45, 0.45, 0.65, 0.65, 0.45, 0.45, 0.45, 0.45],
 [0.45, 0, 0.45, 0.45, 0.45, 0.45, 0.65, 0.45, 0.45, 0.65],
 [0.45, 0.45, 0, 0.45, 0.65, 0.45, 0.45, 0.45, 0.65, 0.45],
 [0.45, 0.45, 0.45, 0, 0.45, 0.45, 0.65, 0.65, 0.45, 0.45],
 [0.65, 0.45, 0.65, 0.45, 0, 0.45, 0.45, 0.45, 0.45, 0.45],
 [0.65, 0.45, 0.45, 0.45, 0.45, 0, 0.45, 0.45, 0.45, 0.65],
 [0.45, 0.65, 0.45, 0.65, 0.45, 0.45, 0, 0.45, 0.45, 0.45],
 [0.45, 0.45, 0.45, 0.65, 0.45, 0.45, 0.45, 0, 0.65, 0.45],
 [0.45, 0.45, 0.65, 0.45, 0.45, 0.45, 0.45, 0.65, 0, 0.45],
 [0.45, 0.65, 0.45, 0.45, 0.45, 0.65, 0.45, 0.45, 0.45, 0]]
 ```

### <font color='8EC044'> **1.2.2 Constructive method** </font>

Now, we need to create our constructive heuristic. This function will construct a path to visit all cities in the problem, based on the probability $p_{ij}$ which is computed unsing only the pheromone trails:
$$p_{ij} := \frac{τ_{ij}}{∑_{z∈S} τ_{iz}} ∀ j \in S$$

Now, it is time to code the function to compute the probability to go from city $i$ to $j$. This function also needs the updated pheromone `trail` and the set of unvisited cities `S`:

In [ ]:
def probability(i, j, trail, S):

  return prob

Let's test your inplementation:

In [ ]:
S = [1,2,3,4]
i = 0
p = []
for j in S:
  p.append(probability(i, j, t_test, S))
p

This is the expected result:

```
[0.225, 0.225, 0.225, 0.325]
```

Now we need to code a constructive method that computes the probability $p_{ij}$. The function is an implementation of this method:

In [ ]:
import random

def constructive(trail):
  n = len(trail)
  unvisited_cities = [i for i in range(n)]
  new_tour = [random.choice(list(unvisited_cities))]
  unvisited_cities.remove(new_tour[0])

  while unvisited_cities:
    if len(unvisited_cities) == 1:
      next_city = unvisited_cities[0]
    else:
      # Compute the cumulative probabilities for all unvisited cities
      # based on the pheromone trials
      prob_dist = [0]

      #insert code here

      # Select a random city based on the probabilities computed
      unif = random.random() # uniform random number between 0 and 1

      # insert code here

    new_tour.append(next_city)
    unvisited_cities.remove(next_city)

  return new_tour

Let's contruct a random path using our brand new constructive method:

In [ ]:
constructive(t_test)

### <font color='8EC044'> **1.2.2 Putting the pieces togheter** </font>

We are ready to create our ACO-TSP implementation, following the previous algorithm.

Let's define the ACO-TSP as a funcion:

In [ ]:
from math import inf

def aco_tsp(distances, num_ants, num_iters, rho, delta, verbose = False):
  n = len(distances)
  trail = [[-1] * n for _ in range(n)]
  trail = init_trail(0.5, trail)
  best_cost = float(inf)
  best_sol = []
  trace = []
  for iter in range(num_iters):
    solutions = []
    costs = []
    for ant in range(num_ants):
      # call the constructive method for all ants
      # append the solution to the solutions list
      # compute and append the cost of the solution in the costs list

      # Insert three code lines here

    best_cost_iter = min(costs)
    best_sol_iter = solutions[costs.index(best_cost_iter)]
    trace.append([iter, best_cost_iter])

    # Update best solution
    if best_cost_iter < best_cost:
      best_cost = best_cost_iter
      best_sol = best_sol_iter
      if verbose:
        print("best sol found at iter %d costs %7.2f" % (iter, best_cost))

    # Update pheromones
    # insert 1 code line here

  if verbose:
    print(best_sol)
    print(trail)
  return best_sol, best_cost, trace

Let's try our ACO-TSP method, solving the prueba10.tsp instance, using just 20  iterations and 50 ants. We are setting the parameters of rho and delta to 0.1 and 0.2, respectively.   

In [ ]:
#Parameters
n_iters = 20
n_ants = 50
rho = 0.1
delta = 0.2

best_sol, best_cost, trace = aco_tsp(distances, n_iters, n_ants, rho, delta, verbose = True)

Let's try to solve a harder intance:

In [ ]:
#Parameters
n_iters = 200
n_ants = 20
rho = 0.1
delta = 0.5

# Read data
data = tools.read_data("berlin52.tsp")
n = len(data)

# Create distances and penalties matrices
distances = tools.calculate_distances(data)

best_sol, best_cost, trace = aco_tsp(distances, n_iters, n_ants, rho, delta)



Print best solution's trajectory

In [ ]:
import pandas as pd
import plotly.express as px


def plot_trace(trace):
  df = pd.DataFrame(trace, columns=['iteration', 'best'])
  fig = px.line(df, x='iteration', y='best', title='Search trajectory')

  return fig

plot_trace(trace)

## <font color='056938'> **1.3 Fast Ant System** </font>

Several authors have pointed the disadvantages of the basic Ant Colony Optimization algorithm [(Taillard, 2022)](https://link.springer.com/chapter/10.1007/978-3-031-13714-3_8). For that reason, FANT, a simplified framework has been proposed.

Let's define the functions needed for implementing the framework.

First, the constructive heuristic:

In [ ]:
def generate_solution_trail(distance, tour, trail):
  n = len(tour)
  for i in range(1, n - 1):
    total =0
    for j in range(i + 1, n):
      total += trail[tour[i - 1]][tour[j]]
    target = unif(0, total - 1)
    j = i
    total = trail[tour[i - 1]][tour[j]]

    while total < target:
      total += trail[tour[i - 1]][tour[j + 1]]
      j += 1
    tour[j], tour[i]= tour[i], tour[j]
  return tour, tools.tsp_distance(distance, tour)

Now we need to create a new function to update the pheromone trials:

In [ ]:
def update_trail(tour, global_best, exploration, exploitation, trail):
  if tour == global_best:
    exploration += 1
    trail = init_trail(exploration, trail)
  else:
    for i in tour:
      n = len(trail[0])
      trail[tour[i]][tour[(i + 1) % n]] += exploration
      trail[global_best[i]][global_best[(i + 1) % n]] += exploitation
  return trail, exploration

In [ ]:
def two_opt_swap(tour, i, j):
    new_tour = tour[:i] + tour[i:j+1][::-1] + tour[j+1:]
    return new_tour

def local_search_tsp(tour, distance):
    n = len(tour)
    best_tour = tour
    best_cost = tools.tsp_distance(distance, tour)
    improved = True

    while improved:
        improved = False
        for i in range(1, n - 2):
            for j in range(i + 2, n):
                new_tour = two_opt_swap(best_tour, i, j)
                new_cost = tools.tsp_distance(distance, new_tour)
                if new_cost < best_cost:
                    best_tour = new_tour
                    best_cost = new_cost
                    improved = True

    return best_tour, best_cost

Let's define the fant function:

In [ ]:
from random_generators import rand_permutation
def tsp_FANT(d, exploitation, iterations):
  n = len(d[0])
  best_cost = float('inf')
  exploration =1
  trail = [[-1] * n for _ in range(n)]
  trail = init_trail(exploration, trail)
  tour = rand_permutation(n)
  trace = []
  for i in range(iterations):
    # build solution
    tour, cost = generate_solution_trail(d, tour, trail)
    # improve built solution witho a local search
    #tour, cost = tsp_LK(d, tour, cost)
    print(cost)
    tour, cost = local_search_tsp(tour, d)
    trace.append([i, cost])
    if cost < best_cost:
      best_cost = cost
      print('\t\t\t\t\tFANT {:d} {:f}'.format(i+1, cost))
      best_sol = list(tour)
      exploration =1
      # Reset exploration to lowest value
      trail = init_trail(exploration, trail)
    else:
      # pheromone trace reinforcement - increase memory
      trail, exploration = update_trail(tour, best_sol, exploration,
                                        exploitation, trail)
  return best_sol, best_cost, trace

In [ ]:
#Parameters
n_iters = 200
exploitation = 1

# Read data
data = tools.read_data("berlin52.tsp")
n = len(data)

# Create distances and penalties matrices
distances = tools.calculate_distances(data)

best_sol, best_cost, trace = tsp_FANT(distances, exploitation, n_iters)

In [ ]:
import pandas as pd
import plotly.express as px


def plot_trace(trace):
  df = pd.DataFrame(trace, columns=['iteration', 'best'])
  fig = px.line(df, x='iteration', y='best', title='Search trajectory')

  return fig

plot_trace(trace)